In [4]:
%system kaggle datasets download -d gpiosenka/cats-in-the-wild-image-classification


["Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /home/averrous/.kaggle/kaggle.json'",
 'Downloading cats-in-the-wild-image-classification.zip to /home/averrous/dev/projects/ml/vision/big-cats-of-the-wild',
 '',
 '  0%|          | 0.00/123M [00:00<?, ?B/s]',
 '  1%|          | 1.00M/123M [00:00<01:53, 1.13MB/s]',
 '  2%|▏         | 2.00M/123M [00:01<01:06, 1.92MB/s]',
 '  2%|▏         | 3.00M/123M [00:01<00:48, 2.62MB/s]',
 '  3%|▎         | 4.00M/123M [00:01<00:35, 3.51MB/s]',
 '  4%|▍         | 5.00M/123M [00:01<00:33, 3.73MB/s]',
 '  5%|▍         | 6.00M/123M [00:02<00:30, 3.97MB/s]',
 '  6%|▌         | 7.00M/123M [00:02<00:25, 4.72MB/s]',
 '  6%|▋         | 8.00M/123M [00:02<00:23, 5.09MB/s]',
 '  7%|▋         | 9.00M/123M [00:02<00:24, 4.90MB/s]',
 '  8%|▊         | 10.0M/123M [00:02<00:21, 5.41MB/s]',
 '  9%|▉         | 11.0M/123M [00:02<00:19, 6.13MB/s]',
 ' 10%|▉         | 12.0M/123M [00:02<00:17, 6.65MB/s]',
 ' 11%|█

In [5]:
%system mv ./cats-in-the-wild-image-classification.zip ./blob


[]

In [ ]:
import os
import zipfile

zip_file = './blob/cats-in-the-wild-image-classification.zip'
zip_ref = zipfile.ZipFile(zip_file, 'r')
zip_ref.extractall('./blob')
zip_ref.close()

In [ ]:
import pandas as pd

data = pd.read_csv('./blob/WILDCATS.CSV')
data.head()

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
data.value_counts('labels')

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimage

In [ ]:
fig = plt.gcf()

fig.set_size_inches(4 * 4, 4 * 4)



for i, img_path in enumerate(data["filepaths"][0:8]):
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    img = mpimage.imread(os.path.join("./blob/",img_path))
    plt.imshow(img)

plt.show()

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
# print("Num GPUs Available: ", tf.config.list_physical_devices("GPU"))
import logging
# tf.get_logger().setLevel(logging.ERROR)
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.compat.v1.Session(config=config)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [ ]:
from keras import layers, Model

In [56]:
img_input = layers.Input(shape=(224, 224, 3))

x = layers.Conv2D(64, 3, activation='relu')(img_input)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(4096, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)
model = Model(img_input, output)


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Optimizer

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01),
              metrics=['acc'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_datagen = ImageDataGenerator(rescale=1./ 255,
                                  #  rotation_range=40,
                                  #  width_shift_range=0.2,
                                  #  height_shift_range=0.2,
                                  #  shear_range=0.2,
                                  #  zoom_range=0.2,
                                  #  horizontal_flip=True
                                  )
validation_datagen = ImageDataGenerator(rescale=1./255)


data_train, data_val = train_test_split(data, test_size=0.2, shuffle=True)

train_generator = train_datagen.flow_from_dataframe(data_train, 
                                                    "./blob", 
                                                    "filepaths", 
                                                    "labels",
                                                    target_size=(224,224),
                                                    batch_size=5,
                                                    class_mode="categorical")
validation_generator = train_datagen.flow_from_dataframe(data_val, 
                                                    "./blob", 
                                                    "filepaths", 
                                                    "labels",
                                                    target_size=(224,224),
                                                    batch_size=5,
                                                    class_mode="categorical")



In [ ]:
train_generator.image_shape

In [ ]:
train_generator.n

In [ ]:
history = model.fit(train_generator,validation_data=validation_generator, steps_per_epoch=350, epochs=80, verbose=1, validation_steps=5)

In [ ]:
result = model.predict(train_generator)

In [ ]:
print(train_generator.filenames[1])
print(train_generator.labels[1])
print(train_generator.class_indices)
result[1]

In [ ]:
import numpy as np
keys = list(train_generator.class_indices.keys())
result_category = list(map(lambda x: keys[np.argmax(x)], result))
print(result_category)